In [ ]:
import requests
import re

def sina_finance_result_parser(data):
    values = data.replace('\n', '').strip().split(';')
    result = {}
    for v in values:
        if len(v) <= 0:
            continue
        tmp = re.split('=|"', v)
        result[tmp[0][11:]] = tmp[2].split(',')
    return result

def parse_option(data):
    return Option()
def get_options(symbols=None):
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'Accept': '*/*',
        'Sec-Fetch-Site': 'cross-site',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Dest': 'script',
        'Referer': 'https://stock.finance.sina.com.cn/',
        'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,zh-TW;q=0.7',
    }

    response = requests.get('https://hq.sinajs.cn/list=' + ','.join(symbols), headers=headers)
    return sina_finance_result_parser(response.text)

def get_option_chain(symbol, year, month):
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'Accept': '*/*',
        'Sec-Fetch-Site': 'cross-site',
        'Sec-Fetch-Mode': 'no-cors',
        'Sec-Fetch-Dest': 'script',
        'Referer': 'https://stock.finance.sina.com.cn/',
        'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8,zh-TW;q=0.7',
    }
    call_list_name = 'OP_UP_{}{}{:02d}'.format(symbol[2:], year, month)
    put_list_name = 'OP_DOWN_{}{}{:02d}'.format(symbol[2:], year, month)
    underlying_name = 's_{}'.format(symbol.lower())
    response = requests.get('https://hq.sinajs.cn/list=' + ','.join([call_list_name, put_list_name, underlying_name]), headers=headers)
    chain_data =  sina_finance_result_parser(response.text)
    return {
        'keys': {
            'underlying':underlying_name,
            'call':call_list_name,
            'put':put_list_name
        },
        'data': chain_data
    }

def get_option_data(symbol, year, month):
    chain = get_option_chain(symbol=symbol, year=year, month=month)
    chain_data = chain['data']
    call_list_name = chain['keys']['call']
    put_list_name = chain['keys']['put']
    options = chain_data[call_list_name] + chain_data[put_list_name] + [chain['keys']['underlying']]
    return get_options(symbols=options)

In [ ]:
options = get_option_data(symbol='SH510300', year=21, month=6)
print(options)

In [ ]:
from a1chemy.common import Option, OptionChain, Underlying
import datetime
import json
import numpy as np
from a1chemy.util.option_greeks_calculator import merton, euro_implied_vol



d = get_options(symbols=['CON_OP_10003321', 'CON_OP_10002879', 's_sh510300'])
#成交量 193 delta:0.9988 Gamma:0.0128 Theta:-0.1876 Vega:0.0056 IV:0.0008 最新价: 0.8400
o = d['CON_OP_10003321']
print(d['s_sh510300'])
print(d['CON_OP_10003321'])
print(len(o))
print(o[43]) # M新合约 A除权后的老合约
print(o[45]) # 期权类型 C/P:Call/Put
print(o[41]) # 成交量
print(datetime.date.fromisoformat(o[46])) # 日期
print(o[7]) # 行权价
print(o[2]) #价格
print(o[50]) #IV
u = Underlying(price=float(d['s_sh510300'][1]))

option = Option(redefine_greeks=True, underlying=u,
                option_type=o[45].lower(), date=datetime.date.fromisoformat(o[46]), strike=float(o[7]), price=float(o[2]))
print(option.option_type)
print(option.price)
print(option.strike)
option.recalculate_greeks(r=0.0241, q=0)
print('iv:{}, delta:{} , gamma:{}, theta:{}, vega:{}, rho:{}'
      .format(option.iv, option.delta, option.gamma, option.theta, option.vega, option.rho))
print(float(np.busday_count( datetime.date.today(), option.date))/240)